In [3]:
import pandas as pd
import numpy as np
import hvplot.pandas

# cadCAD configuration
from cadCAD.configuration.utils import  config_sim
from cadCAD.configuration import Experiment

# cadCAD Engine
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

import numpy as np

In [4]:
# List of all the state variables in the system and their initial values
system_params = {
    'example': [1], # number of created portfolios
}

In [5]:
black_pieces = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
black_pawns = ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
white_pieces = [p.lower() for p in black_pieces]
white_pawns = [p.lower() for p in black_pawns]
empty_row = ['0', '0', '0', '0', '0', '0', '0', '0']

In [6]:
white_pieces

['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']

In [7]:
white_pawns

['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p']

In [8]:
test_row = ['0', '0', '0', '0', 'R', '0', '0', '0']

In [9]:
board = [black_pieces, black_pawns, empty_row, empty_row, test_row, empty_row, white_pawns, white_pieces]

In [10]:
board

[['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', 'R', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']]

In [44]:
board

[['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', 'R', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']]

In [45]:
board[4][4]

'R'

In [11]:
# List of all the state variables in the system and their initial values
initial_state = {
    # State of Portfolios
    'board': board,
    'turn': 'white',
    'finished': False,
    'winner': None,
    'move_number': 1,
    'captured': ({}, {})
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

In [12]:
board[0][0]

'R'

In [13]:
i = 2
j = 5

In [14]:
board[i][j]

'0'

In [16]:
for k in range(7, -1, -1):
    print(k)

7
6
5
4
3
2
1
0


In [32]:
def move_allowed(board, k, j, player):
    piece = board[k][j]
    if piece == '0':
        return -1
    if player == 'white':
        if piece.lower() == piece:
            # This is our piece and we can not move here
            return k
        elif piece.upper() == piece:
            # This is our opponents piece
            return k+1
        else:
            print("We shouldnt end up here.")
            raise
    if player == 'black':
        if piece.lower() == piece:
            # This is our enemy
            return k+1
        elif piece.upper() == piece:
            # This is our piece
            return k
        else:
            print("We shouldnt end up here.")
            raise

In [52]:
def m_rook(board, i, j, player='white'):
    max_up = None
    max_left = None
    max_right = None
    max_down = None
    #up
    for k in range(i-1, -1, -1):
        print(k, j)
        allowed = move_allowed(board, k, j, player)
        print(allowed)
        if allowed == -1:
            continue
        else:
            max_up = allowed
            break
            
    # Left
    for k in range(j-1, -1, -1):
        allowed = move_allowed(board, k, j, player)
        if allowed == -1:
            continue
        else:
            max_left = allowed
            break
            
    # Right
    for k in range(i+1, 8, 1):
#         print(k, j)
        allowed = move_allowed(board, k, j, player)
#         print(allowed)
        if allowed == -1:
            continue
        else:
            max_right = allowed
            break
            
    # Down
    for k in range(i+1, 8, 1):
        allowed = move_allowed(board, k, j, player)
        if allowed == -1:
            continue
        else:
            max_down = allowed
            break
            
    return(max_up, max_left, max_right, max_down)

In [53]:
m_rook(board, 4, 4, 'black')

3 4
-1
2 4
-1
1 4
1


(1, 1, 7, 7)

In [54]:
board

[['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', 'R', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']]

In [49]:
# Left to right a-h
# Bottom to top 1-8

In [5]:
def collude(previous_state):
    return max(np.random.normal(previous_state['colluder_optimality_gap'], 0.1), 0)

def dont_collude(previous_state):
    return max(np.random.normal(previous_state['organic_optimality_gap'], 0.12), 0)

In [7]:
def p_organic_policy(params, substep, state_history, previous_state):
    organic_distribution = dont_collude(previous_state)
    return {'organic_distribution': organic_distribution}

def p_colluder_policy(params, substep, state_history, previous_state):
    colluder_distribution = collude(previous_state)
    return {'colluder_distribution': colluder_distribution}

In [9]:
def s_organic_distribution(params, substep, state_history, previous_state, policy_input):
    key = 'organic_distribution'
    value = policy_input[key]
    return (key, value)

def s_colluder_distribution(params, substep, state_history, previous_state, policy_input):
    key = 'colluder_distribution'
    value = policy_input[key]
    return (key, value)

In [10]:
partial_state_update_blocks = [
    {
        'policies': {
            'organic_policy': p_organic_policy,
            'colluder_policy': p_colluder_policy,

        },
        'variables': {
            'organic_distribution': s_organic_distribution,
            'colluder_distribution': s_colluder_distribution,

        }
    }
]

In [11]:
del configs[:]

sim_config = config_sim({
    'N': 10, # Number of Monte Carlo Runs
    'T': range(100), # Number of timesteps
    'M': system_params,
})
experiment = Experiment()
experiment.append_configs(
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks,
    sim_configs=sim_config,
)

In [12]:
exec_context = ExecutionContext()
simulation = Executor(
    exec_context=exec_context,
    configs=configs,
)

In [13]:
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 2, 10, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SubsetIDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Ns       : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
ExpIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 0.16s


In [14]:
df = pd.DataFrame(raw_result)
df

,organic_optimality_gap,colluder_optimality_gap,simulation,subset,run,substep,timestep,organic_distribution,colluder_distribution
0,0.5,0.3,0,0,1,0,0,NaN,NaN
1,0.5,0.3,0,0,1,1,1,0.366867,0.217916
2,0.5,0.3,0,0,1,1,2,0.395652,0.356852
3,0.5,0.3,0,0,1,1,3,0.564924,0.357972
4,0.5,0.3,0,0,1,1,4,0.603976,0.252642
...,...,...,...,...,...,...,...,...,...
1005,0.5,0.3,0,0,10,1,96,0.604110,0.398127
1006,0.5,0.3,0,0,10,1,97,0.591379,0.260129
1007,0.5,0.3,0,0,10,1,98,0.414653,0.440853
1008,0.5,0.3,0,0,10,1,99,0.676922,0.420930


In [15]:
df.hvplot.kde(['colluder_distribution', 'organic_distribution'], title='Optimality Gap Distribution', groupby='run', xlim=(0,1))

:DynamicMap   [run]
   :NdOverlay   [Variable]
      :Distribution   [value]   (Density)

In [13]:
df

,organic_optimality_gap,colluder_optimality_gap,simulation,subset,run,substep,timestep,organic_distribution,colluder_distribution
0,0.5,0.3,0,0,1,0,0,NaN,NaN
1,0.5,0.3,0,0,1,1,1,0.384892,0.314623
2,0.5,0.3,0,0,1,1,2,0.448249,0.387461
3,0.5,0.3,0,0,1,1,3,0.525539,0.326186
4,0.5,0.3,0,0,1,1,4,0.387261,0.265082
...,...,...,...,...,...,...,...,...,...
1005,0.5,0.3,0,0,10,1,96,0.600193,0.260825
1006,0.5,0.3,0,0,10,1,97,0.705680,0.417178
1007,0.5,0.3,0,0,10,1,98,0.284489,0.400643
1008,0.5,0.3,0,0,10,1,99,0.560108,0.364641


In [16]:
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import PPO
from gym import spaces
import gym
import json

In [17]:

class Environment(gym.Env):
    def __init__(self):

        self.action_space = spaces.Discrete(2)

        self.observation_space = spaces.Discrete(2)
        
        self.performance = []
        
        self.reset()
        
    def _next_observation(self):
        return 1
    
    
    def _take_action(self, action):
        if action == 1:
            reward = collude(initial_state)
        else:
            reward = dont_collude(initial_state)
        return reward
        

    def reset(self):
        # Return the next observation
        return self._next_observation()
    
    def step(self, action):
        # Execute one time step within the environment
        reward = self._take_action(action)  
        done = 0
        obs = self._next_observation()  
        self.performance.append([
            reward,
        ])
        return obs, reward, done, {}
    
    def render(self):
        pass
#         print(f"Reward: {self.performance}")

In [18]:
env = Environment()
agent = PPO(MlpPolicy, env, verbose=0)
agent.learn(total_timesteps=2000)

In [19]:
obs = env.reset()
for i in range(2000):
    action, _states = agent.predict(obs)
    obs, rewards, done, info = env.step(action)
    if not i % 200:
        env.render()

In [20]:
performance = pd.DataFrame(env.performance)

In [21]:
performance.hvplot.kde()

:Distribution   [0]   (Density)